# file explaination

## StyleDataset.py

In [ ]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import glob

mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]

class StyleDataset(Dataset):
    def __init__(self, root):
        self.pics = glob.glob(f"{root}/*.jpg")
        self.tf = transforms.Compose([
            transforms.Resize(512),
            transforms.CenterCrop(512),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)

        ])
    def __len__(self): return len(self.pics)
    def __getitem__(self, i):
        img = Image.open(self.pics[i]).convert('RGB')
        return self.tf(img)

TrainLoader = DataLoader(StyleDataset("data/train"), batch_size=4, shuffle=True, num_workers=4, pin_memory=True)
ValidateLoader = DataLoader(StyleDataset("data/val"), batch_size=4, shuffle=True, num_workers=4, pin_memory=True)

数据处理用Image打开，转化为RGB格式，然后用transforms转换，做转化，裁切，转化为张量，归一化，然后加载到DataLoader中。
DataLoader参数包括Dataset的类，每次加载的数量，num_workers是线程数，pin_memory是是否固定在显存中。

## rtst.py

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        super(ConvLayer, self).__init__()
        padding = kernel_size // 2
        self.conv = nn.Sequential(
            nn.ReflectionPad2d(padding=padding),
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride),
            nn.InstanceNorm2d(out_channels, affine=True),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.ReflectionPad2d(padding=1),
            nn.Conv2d(in_channels=channels, out_channels=channels, kernel_size=3, stride=1),
            nn.InstanceNorm2d(channels, affine=True),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(padding=1),
            nn.Conv2d(in_channels=channels, out_channels=channels, kernel_size=3, stride=1),
            nn.InstanceNorm2d(channels, affine=True),
        )
    
    def forward(self, x):
        return x + self.block(x)

class UpsampleConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, upsample=None):
        super(UpsampleConvLayer, self).__init__()
        self.upsample = upsample
        padding = kernel_size // 2
        self.conv = nn.Sequential(
            nn.ReflectionPad2d(padding=padding),
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride),
            nn.InstanceNorm2d(out_channels, affine=True),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        if self.upsample:
            F.interpolate(input=x, scale_factor=self.upsample, mode="nearest")
        return self.conv(x)

class TransformerNet(nn.Module):
    def __init__(self):
        super(TransformerNet, self).__init__()

        self.conv1 = ConvLayer(3, 32, kernel_size=9, stride=1)
        self.conv2 = ConvLayer(32, 64, kernel_size=3, stride=2)
        self.conv3 = ConvLayer(64, 128, kernel_size=3, stride=2)

        self.res1 = ResidualBlock(128)
        self.res2 = ResidualBlock(128)
        self.res3 = ResidualBlock(128)
        self.res4 = ResidualBlock(128)
        self.res5 = ResidualBlock(128)

        self.deconv1 = UpsampleConvLayer(128, 64, kernel_size=3, stride=1, upsample=2)
        self.deconv2 = UpsampleConvLayer(64, 32, kernel_size=3, stride=1, upsample=2)
        self.deconv3 = nn.Sequential(
            nn.ReflectionPad2d(4),
            nn.Conv2d(in_channels=32, out_channels=3, kernel_size=9, stride=1),
            nn.Tanh()
        )

    def forward(self, x):
        y = self.conv1(x)
        y = self.conv2(y)
        y = self.conv3(y)
        y = self.res1(y)
        y = self.res2(y)
        y = self.res3(y)
        y = self.res4(y)
        y = self.res5(y)
        y = self.deconv1(y)
        y = self.deconv2(y)
        y = self.deconv3(y)
        return y

if __name__ == "__main__":
    model = TransformerNet()
    random_pic = torch.randn(1,3,128,128)
    output = model(random_pic)
    print(f"output: {output.shape}")



首先使用三个卷积层扩展到128，长宽压缩四倍，然后用ResBlock处理5层，然后两层上采样使图片尺寸还原，最后一层压缩到3，对应RGB。

## fe_model.py

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from torchvision.models import vgg19
#feature extracting model
fe_model = vgg19(weights="IMAGENET1K_V1").features.eval().to(device=device)
for param in fe_model.parameters():
    param.requires_grad=False

from torchvision.models.feature_extraction import create_feature_extractor 

#这里选择的层数是超参数，可以自己调节
content_layers = ['21']
style_layers = ['0', '5', '10', '19', '28']
extractor = create_feature_extractor(fe_model, return_nodes={**{l: l for l in content_layers}, **{l: l for l in style_layers}})


获取torchvision.models中的vgg19，然后设置权重为IMAGENET1K_V1，用features获取前面的卷积提取特征部分，然后调到评测模式，放置到设备，然后冻结参数，提取特定层的特征。